In [1]:
import os
import pickle
import numpy as np
from keras.models import Sequential
import keras.layers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train, valid, test = pickle.load(open('dataset.pickle', 'rb'))
train = train.fillna(0)
valid = valid.fillna(0)
test = test.fillna(0)

In [3]:
#x_train = train[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_train = train.iloc[:,-8:]
#x_valid = valid[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_valid = valid.iloc[:,-8:]
#x_test = test[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
#y_test = np.asarray(test.iloc[:,-8:])
#y_test_decode = []
#for i in range(y_test.shape[0]):
#    y_test_decode.insert(len(y_test_decode),np.argmax(y_test[i,:])#)

In [4]:
scaler = MinMaxScaler() #scale features between 0 and 1
#x_train = scaler.fit_transform(x_train)
#x_train = np.asarray(x_train).reshape(x_train.shape[0], 1, x_train.shape[1])
#x_valid = scaler.fit_transform(x_valid)
#x_test = scaler.fit_transform(x_test)
#x_test = np.asarray(x_test).reshape(x_test.shape[0], 1, x_test.shape[1])

In [5]:
# fit model on dataset
def fit_model(x_train, y_train, x_valid, y_valid):
    model = Sequential()
    #model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2]), bias_regularizer=L1L2(l1=0.01, l2=0.01)))
    #model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2])))
    #model.add(Dense(2000, activation='relu'))
    model.add(Dense(200, input_dim=x_train.shape[1], activation='relu'))
    model.add(Dropout(0.2, noise_shape=None, seed=None))
    model.add(BatchNormalization())
    model.add(Dense(50, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(y_train.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.fit([x_train], batch_size=20, y=y_train, verbose=1, validation_split=0.25, shuffle=True, epochs=1)
    model.fit([x_train], batch_size=100, y=y_train, verbose=1, validation_data=(x_valid, y_valid), shuffle=True, epochs=20)
    return model

In [6]:
def get_x_data(fields):
    x_train = train[fields]
    x_train = scaler.fit_transform(x_train)
    x_valid = valid[fields]
    x_valid = scaler.fit_transform(x_valid)
    x_test = test[fields]
    x_test = scaler.fit_transform(x_test)
    return (x_train, x_valid, x_test)

In [7]:
# create directory for models
dir_name = "nn_ensemble2"
if not os.path.exists(dir_name):
    os.makedirs(dir_name)
# fit and save models
target_fields = ['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']
for i in range(len(target_fields)):
    # fit model
    x_data = get_x_data(target_fields[i])
    x_train = x_data[0]
    x_valid = x_data[1]
    model = fit_model(x_train, y_train, x_valid, y_valid)
    # save model
    filename = dir_name + '/model_' + str(i + 1) + '.nn'
    model.save(filename)
    print('Saved %s' % filename)

Train on 297773 samples, validate on 34188 samples
Epoch 1/20
297773/297773 [==============================] - 12s 41us/step - loss: 1.4081 - acc: 0.4544 - val_loss: 1.7535 - val_acc: 0.3578
Epoch 2/20
297773/297773 [==============================] - 12s 39us/step - loss: 1.3465 - acc: 0.4741 - val_loss: 1.6022 - val_acc: 0.3163
Epoch 3/20
297773/297773 [==============================] - 11s 37us/step - loss: 1.3338 - acc: 0.4802 - val_loss: 1.7236 - val_acc: 0.2754
Epoch 4/20
297773/297773 [==============================] - 11s 36us/step - loss: 1.3230 - acc: 0.4856 - val_loss: 2.1673 - val_acc: 0.2151
Epoch 5/20
297773/297773 [==============================] - 11s 36us/step - loss: 1.3169 - acc: 0.4894 - val_loss: 1.8071 - val_acc: 0.2646
Epoch 6/20
297773/297773 [==============================] - 11s 37us/step - loss: 1.3095 - acc: 0.4927 - val_loss: 1.9344 - val_acc: 0.3014
Epoch 7/20
297773/297773 [==============================] - 11s 36us/step - loss: 1.3043 - acc: 0.4963 - val_

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


Train on 297773 samples, validate on 34188 samples
Epoch 1/20
297773/297773 [==============================] - 12s 39us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 2/20
297773/297773 [==============================] - 11s 37us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 3/20
297773/297773 [==============================] - 11s 37us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 4/20
297773/297773 [==============================] - 11s 37us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 5/20
297773/297773 [==============================] - 11s 37us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 6/20
297773/297773 [==============================] - 11s 37us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 7/20
297773/297773 [==============================] - 11s 37us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 8/20


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


Train on 297773 samples, validate on 34188 samples
Epoch 1/20
297773/297773 [==============================] - 11s 38us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 2/20
297773/297773 [==============================] - 10s 35us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 3/20
297773/297773 [==============================] - 11s 35us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 4/20
297773/297773 [==============================] - 11s 36us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 5/20
297773/297773 [==============================] - 10s 35us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 6/20
297773/297773 [==============================] - 10s 35us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 7/20
297773/297773 [==============================] - 10s 35us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 8/20


...

Train on 297773 samples, validate on 34188 samples
Epoch 1/20
297773/297773 [==============================] - 12s 39us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 2/20
297773/297773 [==============================] - 11s 35us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 3/20
297773/297773 [==============================] - 11s 35us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 4/20
297773/297773 [==============================] - 11s 35us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 5/20
297773/297773 [==============================] - 10s 35us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 6/20
297773/297773 [==============================] - 10s 35us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 7/20
297773/297773 [==============================] - 11s 35us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 8/20


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


Train on 297773 samples, validate on 34188 samples
Epoch 1/20
297773/297773 [==============================] - 12s 40us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 2/20
297773/297773 [==============================] - 11s 35us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 3/20
297773/297773 [==============================] - 11s 35us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 4/20
297773/297773 [==============================] - 11s 36us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 5/20
297773/297773 [==============================] - 11s 35us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 6/20
297773/297773 [==============================] - 11s 36us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 7/20
297773/297773 [==============================] - 11s 36us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 8/20


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


Train on 297773 samples, validate on 34188 samples
Epoch 1/20
297773/297773 [==============================] - 12s 41us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 2/20
297773/297773 [==============================] - 11s 37us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 3/20
297773/297773 [==============================] - 11s 37us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 4/20
297773/297773 [==============================] - 11s 36us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 5/20
297773/297773 [==============================] - 11s 36us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 6/20
297773/297773 [==============================] - 11s 36us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 7/20
297773/297773 [==============================] - 11s 37us/step - loss: nan - acc: 0.2744 - val_loss: nan - val_acc: 0.2493
Epoch 8/20


In [ ]:
target_fields = ['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']
x_data = get_x_data(target_fields[0])
print(x_data[0].shape)
print(len(target_fields))